This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'ZbunUMxjexjYt8DyhESA'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json
print('Loading Libraries')

Loading Libraries


In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
stock_exchange = 'FSE'
ticker = 'AFX_X'
data_format = 'json'
domain = 'https://www.quandl.com/api/v3/datasets'
params = {
    'api_key': API_KEY,
    'start_date': '2017-01-01',
    'end_date': '2017-12-31',
    }
url = 'https://www.quandl.com/api/v3/datasets/%s/%s.%s?'%(stock_exchange, ticker, data_format)
response = requests.get(url, params=params)
print(response)
data = response.json()

<Response [200]>


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(data.keys())
print(len(data))
print(type(data['dataset']))



dict_keys(['dataset'])
1
<class 'dict'>


In [6]:
data = data['dataset'] # for simplicity

In [7]:
print(data.keys())
print(data['column_names'])

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [ ]:
# # date open high low close change tradedvolume
# 0,   1,   2,   3,  4,    5,     6

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
prices = data['data']
num_periods = len(prices)

In [9]:
# initial variables
max_open, min_open = -9999, 9999
max_change_intraday = -99; # change = high - low
max_change_nextday = -99; # close - close (previous)
previous_close = prices[0][3]
sum_vol = 0; # for finding average
avg_count = 0;
temp_change = 0.0;

In [10]:

for i in range(num_periods):
    # finding max/min
    if prices[i][1] != None:
        if prices[i][1] > max_open:
            max_open = prices[i][1]
        if prices[i][1] < min_open:
            min_open = prices[i][1]
    
    # finding max intraday change
    if ((prices[i][2] != None) & (prices[i][3] != None)):
        if (prices[i][2] - prices[i][3]) > max_change_intraday:
            max_change_intraday = (prices[i][2] - prices[i][3])
    
    # finding max close-change
    if ((prices[i][3] != None) & (previous_close != None)):
        if abs(prices[i][4] - previous_close) > max_change_nextday:
            max_change_nextday = abs(prices[i][4] - previous_close)
    previous_close = prices[i][4]
    
    # finding average volume
    if ((prices[i][6]) != None):
        sum_vol = sum_vol + prices[i][6]
        avg_count+=1
        
avg_vol = sum_vol/avg_count
print('\n\n')
print('The max and open prices are %.2f and %.2f'%(max_open, min_open))
print('The max intraday change (high - low) is %.2f'%max_change_intraday)
print('The absolute max change in change in price next day: %.2f'%max_change_nextday)
print('The average trading volume: %i'%avg_vol)




The max and open prices are 53.11 and 34.00
The max intraday change (high - low) is 2.81
The absolute max change in change in price next day: 2.56
The average trading volume: 89124


In [11]:
# to find the median we need to sort the volume. Extracting volume in a separate list vol (without any Nones)
vol = [None]*avg_count # initialize space to store volume
k = 0
for i in range(num_periods): # grabbing the volume and putting in separate list
    if (prices[i][6] != None):
        vol[k] = prices[i][6]
        k+=1


In [13]:
# %% create our own sorting function - merge-sort algorithm
# two fast algorithms - quicksort or mergesort -- let's do mergesort (easier for me to understand)
def mergeSort(array):
    if len(array) > 1:
        index = int(len(array)*0.5) # finding mid point
        # splitting into two halves
        left = array[:index] 
        right = array[index:]
        
        # initialize counters for comparison
        # i is for left half
        # j is for right half
        # k is for array when merged
        i, j, k = 0,0,0

        # recursive. Keep halving until down to duples 
        mergeSort(left)
        mergeSort(right)
        
        # within each halves-subsections
        # the sorting processing
        while (i<len(left)) & (j<len(right)):
            if left[i] < right[j]: #compare
                array[k] = left[i] # filling out the merged
                i+=1
            else:
                array[k] = right[j]
                j+=1
            k+=1
        
        # filling out leftover halves (assuming i,j didn't finish in the comparison loop above)
        while (i<len(left)):
            array[k] = left[i]
            i+=1
            k+=1
        
        while (j<len(right)):
            array[k] = right[j]
            j+=1
            k+=1       

In [14]:
# %% Calculate median
mergeSort(vol)
# vol.sort() if we wanted to use python's built in sort
if avg_count%2 == 0: # if number of records is even, take average
    median = (vol[avg_count/2] + vol[avg_count/2-1])/2
else: 
    median = vol[int(avg_count/2)]

print('The median: %i'%median)


The median: 76286
